In [1]:
# Import Libs
import time
init_time = time.time()

import os
import pickle
import numpy as np
import ROOT

from functions import HistogramFunctions,AuxiliarFunctions,FigureFunctions,FitFunctions,rootnotes

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['lines.linewidth'] = 2
plt.rcParams['legend.handlelength'] = 3
plt.rcParams['legend.borderpad'] = 0.3
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['ytick.labelsize'] = 20

current_time = time.time()
print()'Time to import libraries: %1.3f seconds'%(current_time-init_time))


SyntaxError: invalid syntax (<ipython-input-1-b11261f9599e>, line 24)

In [ ]:
data_path = '/home/micael/MyWorkspace/multi_Higgs_classifier/data_inputs'

signal = np.load(data_path+'lvbb125_data_array.npy')
signal = signal[:,:23]
signal = np.delete(signal, 11, 1)
sgn_trgt_unbalanced = np.ones(signal.shape[0], dtype=np.int)

bkgs  = {}
targets = {}
for ibk in bkg_names:
    print('Creating the dict with {} bkg data and the targets...'.format(ibk))
    targets[ibk] = {}
    bkgs[ibk] = np.load(data_path+ibk+'_data_array.npy')
    bkgs[ibk]  = bkgs[ibk][:,:23]
    bkgs[ibk] = np.delete(bkgs[ibk], 11, 1)
    
    bkg_trgt_balanced = np.zeros(signal.shape[0], dtype=np.int)
    bkg_trgt_unbalanced = np.zeros(bkgs[ibk].shape[0], dtype=np.int)
    targets[ibk]['unbalanced'] = np.append(sgn_trgt_unbalanced, bkg_trgt_unbalanced)
    targets[ibk]['balanced'] = np.append(sgn_trgt_unbalanced, bkg_trgt_balanced)
    
def get_models(raw_classifiers):
    models = {}
    for ifold in raw_classifiers.keys():
        models[ifold] = {}
        for jinit in raw_classifiers[ifold].keys():
            if jinit == 'best init' or jinit == 'pre_proc':
                continue
            tmp = raw_classifiers[ifold][jinit]['model']
            models[ifold][jinit] = model_from_json(json.dumps(json.loads(tmp), separators=(',',':')))
            models[ifold][jinit].set_weights(raw_classifiers[ifold][jinit]['weights'])
    return models

In [ ]:
pickle_path = '/home/micael/MyWorkspace/multi_Higgs_classifier/classificadores/'
bkg_names = ['TTbar', 'Wbb']
classifiers = {}

for bkg in bkg_names:
    with open(pickle_path+bkg+'_classificadores.10sorts.10inits.mapstd.mse.subsampling.batchsize1024.earlystopping50.pkl', 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        classifiers[bkg] = u.load()

classifiers_noPtRel = {}

for bkg in bkg_names:
    with open(pickle_path+bkg+'_classificadores.noPtRel.10sorts.10inits.mapstd.mse.subsampling.batchsize1024.earlystopping50.pkl', 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        classifiers_noPtRel[bkg] = u.load()

In [ ]:
models = {}
for imodel in classifiers.keys():
    models[imodel] = get_models(classifiers[imodel][10])
    
models_noPtRel = {}
for imodel in classifiers_noPtRel.keys():
    models_noPtRel[imodel] = get_models(classifiers_noPtRel[imodel][10])

In [ ]:
scores

In [ ]:
roc_auc_score(sparse_trgt[test_id], scores)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
scores = classifiers[0].predict(norm_data[test_id])
scores.shape
fpr, tpr, thresholds=roc_curve(sparse_trgt[test_id][:,1], scores[:,1])
fpr2, tpr2, thresholds2=roc_curve(sparse_trgt[test_id][:,0], scores[:,0])

In [ ]:
plt.plot(fpr,tpr)
plt.plot(fpr2,tpr2)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix

data_labels = ['Bkg','Signal']
plt.rcParams['font.weight'] = 'bold'

choose_fold = 0

train_id, test_id = CVO[choose_fold]
# normalize data based in train set
if norm == 'mapstd':
    scaler = preprocessing.StandardScaler().fit(data[train_id,:])
elif norm == 'mapstd_rob':
    scaler = preprocessing.RobustScaler().fit(data[train_id,:])
elif norm == 'mapminmax':
    scaler = preprocessing.MinMaxScaler().fit(data[train_id,:])
        
norm_data = scaler.transform(data)

output = classifiers[choose_fold].predict(norm_data)
output_class = np.argmax(output,axis=1)

cm = confusion_matrix(trgt[train_id], output_class[train_id])
cm_data = cm
cm = cm.astype('float')/ cm.sum(axis=1)[:, np.newaxis]
cm_normalized = 100.*cm

fig = plt.figure(figsize=(13,10))
ax = fig.add_subplot(111)
ax.set_aspect(1)

im = ax.imshow(cm_normalized, interpolation='nearest', cmap=plt.cm.Greys,clim=(0.0, 100.0))

width, height = cm_normalized.shape


for x in xrange(width):
    for y in xrange(height):
        if cm_normalized[x][y] < 50.:
            ax.annotate('%1.3f%%\n(%i)'%(cm_normalized[x][y],cm_data[x][y]), xy=(y, x),
                        horizontalalignment='center',
                        verticalalignment='center',fontsize=20)
        else:
            ax.annotate('%1.3f%%\n(%i)'%(cm_normalized[x][y],cm_data[x][y]), xy=(y, x),
                        horizontalalignment='center',
                        verticalalignment='center',color='white',fontsize=20)

ax.set_title('Confusion Matrix Train Data',fontweight='bold',fontsize=15)
fig.colorbar(im)

tick_marks = np.arange(len(data_labels))
ax.xaxis.set_ticks(tick_marks)
ax.xaxis.set_ticklabels(data_labels)

ax.yaxis.set_ticks(tick_marks)
ax.yaxis.set_ticklabels(data_labels)

ax.set_ylabel('True Label',fontweight='bold',fontsize=15)
ax.set_xlabel('Predicted Label',fontweight='bold',fontsize=15)

#save image
fig.savefig(save_point+'/'+'matrix_confusao_encontro_fisica_conjunto_treinamento_com_todos_backgrounds.pdf')

#plt.savefig(path_tosave+'/'+'ConfusionMatrixClassfinal_12clustersTEST_ResultnPtRel.pdf')
#fig.savefig(path_tosave+'/'+'ConfusionMatrixClassfinal_5clustersttbar-4clustersignalTEST_BACKUP.pdf')

In [ ]:
from sklearn import metrics

round(metrics.silhouette_score(norm_data[test_id],output_class[test_id]),3)

In [ ]:
from scipy.stats.mstats import gmean


def Acc(output,true_label):
    acc = np.zeros(len(np.unique(output)))
    for i in range(len(np.unique(output))):
        acc[i] = float(np.sum(output[true_label==i]==i))/float((np.sum(true_label==i)))
    return acc

# Calculate the sum-product
def SP(output,true_label):
    
    SP = np.power(np.mean(Acc(output,true_label))*gmean(Acc(output,true_label)),0.5)
    
    return SP

In [ ]:
round(metrics.accuracy_score(y_pred=output_class[test_id],y_true=trgt[test_id]),3)

In [ ]:
norm_signal = scaler.transform(signal)

output_signal = classifiers[choose_fold].predict(norm_signal)
output_signal_class = np.argmax(output_signal,axis=1)

#print 'output_sinal',output_signal

In [ ]:
ROOT.gROOT.Reset()
ROOT.gStyle.SetOptStat(0)

In [ ]:
import ROOT
ROOT.gROOT.Reset()
ROOT.gStyle.SetOptStat(0)

# for histograms
bins = 60
m_max = 300
m_min = 0

ylabel = "Occurrences"

# Use Colors
list_color_TTbar = []
list_color_Wbb = []

list_color_TTbar.append(ROOT.kRed) # For Signal
list_color_Wbb.append(ROOT.kRed) # For Signal

list_color_Wbb.append(ROOT.kGreen+3) # For Wbb
#list_color.append(ROOT.kGreen+2) # For Wll
#list_color.append(ROOT.kGreen+1) # For Wcc
#list_color.append(ROOT.kGreen-6) # For Wcl
#list_color.append(ROOT.kGreen-9) # For Wll
list_color_TTbar.append(ROOT.kOrange+0) # For TTbar
#list_color.append(ROOT.kGray+3) # For WW
#list_color.append(ROOT.kGray) # For WZ
#list_color.append(ROOT.kGray+1) # For ZZ

#  Make a list of mass histogram
list_inf_M_TTbar = []
list_inf_M_Wbb = []

hist_inf = HistogramFunctions.OneDimHistInfo("Signal Mass ","h_lvbb_M",bins,m_min,m_max,"Invariant Mass",ylabel)
list_inf_M_TTbar.append(hist_inf)
list_inf_M_Wbb.append(hist_inf)


hist_inf = HistogramFunctions.OneDimHistInfo("Bkg Wbb Mass ","h_Wbb_M",bins,m_min,m_max,"Invariant Mass",ylabel)
list_inf_M_Wbb.append(hist_inf)

#hist_inf = HistogramFunctions.OneDimHistInfo("Bkg Wbl Mass ","h_Wbl_M",bins,m_min,m_max,"Invariant Mass",ylabel)
#list_inf_M.append(hist_inf)

#hist_inf = HistogramFunctions.OneDimHistInfo("Bkg Wcc Mass ","h_Wcc_M",bins,m_min,m_max,"Invariant Mass",ylabel)
#list_inf_M.append(hist_inf)

#hist_inf = HistogramFunctions.OneDimHistInfo("Bkg Wcl Mass ","h_Wcl_M",bins,m_min,m_max,"Invariant Mass",ylabel)
#list_inf_M.append(hist_inf)

#hist_inf = HistogramFunctions.OneDimHistInfo("Bkg Wll Mass ","h_Wll_M",bins,m_min,m_max,"Invariant Mass",ylabel)
#list_inf_M.append(hist_inf)

hist_inf = HistogramFunctions.OneDimHistInfo("Bkg TTbar","h_TTbar_M",bins,m_min,m_max,"Invariant Mass",ylabel)
list_inf_M_TTbar.append(hist_inf)

#hist_inf = HistogramFunctions.OneDimHistInfo("Bkg WW","h_WW_M",bins,m_min,m_max,"Invariant Mass",ylabel)
#list_inf_M.append(hist_inf)

#hist_inf = HistogramFunctions.OneDimHistInfo("Bkg WZ","h_WZ_M",bins,m_min,m_max,"Invariant Mass",ylabel)
#list_inf_M.append(hist_inf)

#hist_inf = HistogramFunctions.OneDimHistInfo("Bkg ZZ","h_ZZ_M",bins,m_min,m_max,"Invariant Mass",ylabel)
#list_inf_M.append(hist_inf)

list_hist_M_TTbar = HistogramFunctions.CreateListOf1DHistograms(list_inf_M_TTbar,list_color_TTbar)
list_hist_M_Wbb = HistogramFunctions.CreateListOf1DHistograms(list_inf_M_Wbb,list_color_Wbb)
c = rootnotes.canvas("c", (800, 600))
d = rootnotes.canvas("d", (800, 600))

list_hist_M_TTbar,list_hist_M_Wbb

In [ ]:
from sklearn.externals import joblib
#Use ROOT in order to create a numpy array with selected variables


processes = "lvbb125,TTbar,Wbb"#,Wbl,Wll,Wcl,Wcc,WW,WZ,ZZ"
#processes = "lvbb125,ZZ,TTbar"
#output_path = "/home/isabella/Workspace/python/PtRelAnalysis/PtRelAnalysis/Results"
output_path = "/home/micael/MyWorkspace/multi_Higgs_classifier/plots_v2"

data_path = "/home/micael/MyWorkspace/DATA/ROOT_Files"

for process in processes.split(","):
    data_input = data_path+"/"+process+".root"
    treeName = "perevent"
    file = ROOT.TFile(data_input,"READ")
    tree = file.Get(treeName)
    #nrEvents = tree.GetEntries()
    #nrEvents = 94999

    for (i,entry) in enumerate(tree):
        if nrEvents>0:
            if i>nrEvents:
                break
                
        #=============================================================================
        #=======================EMJESGSCMu========================================
        # variables for EMJESGSCMu j1.     
        E1_EMJESGSCMu = tree.j1_EMJESGSCMu_E
        Phi1_EMJESGSCMu = tree.j1_EMJESGSCMu_Phi
        Eta1_EMJESGSCMu = tree.j1_EMJESGSCMu_Eta
        Pt1_EMJESGSCMu = tree.j1_EMJESGSCMu_Pt
    
        # variables for EMJESGSCMu j2.     
        E2_EMJESGSCMu = tree.j2_EMJESGSCMu_E
        Phi2_EMJESGSCMu = tree.j2_EMJESGSCMu_Phi
        Eta2_EMJESGSCMu = tree.j2_EMJESGSCMu_Eta
        Pt2_EMJESGSCMu = tree.j2_EMJESGSCMu_Pt
    
        # PtRel in EMJESGSCMu
        Theta1_EMJESGSCMu = AuxiliarFunctions.theta(Eta1_EMJESGSCMu)
        Comp1_EMJESGSCMu = AuxiliarFunctions.Componets(E1_EMJESGSCMu,Eta1_EMJESGSCMu,Phi1_EMJESGSCMu,Theta1_EMJESGSCMu)
        norm1_EMJESGSCMu = AuxiliarFunctions.norm(Comp1_EMJESGSCMu)
    
        Theta2_EMJESGSCMu = AuxiliarFunctions.theta(Eta2_EMJESGSCMu)
        Comp2_EMJESGSCMu = AuxiliarFunctions.Componets(E2_EMJESGSCMu,Eta2_EMJESGSCMu,Phi2_EMJESGSCMu,Theta2_EMJESGSCMu)
        norm2_EMJESGCMu = AuxiliarFunctions.norm(Comp2_EMJESGSCMu)
    
        # sum of components.
        Comp3_EMJESGSCMu = AuxiliarFunctions.somaComponents(Comp1_EMJESGSCMu,Comp2_EMJESGSCMu)
        norm3_EMJESGSCMu = AuxiliarFunctions.norm(Comp3_EMJESGSCMu)
        numerador1_EMJESGSCMu = AuxiliarFunctions.numerador(Comp1_EMJESGSCMu,Comp3_EMJESGSCMu)    
        angle1_EMJESGSCMu = AuxiliarFunctions.alpha(numerador1_EMJESGSCMu,norm1_EMJESGSCMu,norm3_EMJESGSCMu)
    
        #compute Relative Pt
        PtRel_j1_EMJESGSCMu = AuxiliarFunctions.PtRelative(E1_EMJESGSCMu,angle1_EMJESGSCMu)
    
        #==========================
        #==========================
    
        # only for EM and j1.
        j1_FracEM3 = tree.j1_FracEM3
        j1_FracTile0 = tree.j1_FracTile0
        j1_TrkWidth = tree.j1_TrkWidth
        j1_EMF = tree.j1_EMF
        j1_JVF = tree.j1_JVF
        j1_NTrk = tree.j1_NTrk 
        j1_SumPtTrk = tree.j1_SumPtTrk
        # only for EM and j2.
        j2_FracEM3 = tree.j2_FracEM3
        j2_FracTile0 = tree.j2_FracTile0
        j2_TrkWidth = tree.j2_TrkWidth
        j2_EMF = tree.j2_EMF
        j2_JVF = tree.j2_JVF
        j2_NTrk = tree.j2_NTrk 
        j2_SumPtTrk = tree.j2_SumPtTrk
    
        
        NNinput = np.array([j1_FracEM3,j1_FracTile0,j1_TrkWidth,j1_EMF,j1_JVF,j1_NTrk,j1_SumPtTrk,Pt1_EMJESGSCMu,Eta1_EMJESGSCMu, Phi1_EMJESGSCMu, E1_EMJESGSCMu, PtRel_j1_EMJESGSCMu,
                            j2_FracEM3,j2_FracTile0,j2_TrkWidth,j2_EMF,j2_JVF,j2_NTrk,j2_SumPtTrk,Pt2_EMJESGSCMu,Eta2_EMJESGSCMu, Phi2_EMJESGSCMu, E2_EMJESGSCMu, PtRel_j1_EMJESGSCMu])
        
        V_NNinput = np.reshape(NNinput, (1,24))
        norm_NNinput = scaler.transform(V_NNinput)
        
        output_NN = classifiers[choose_fold].predict(norm_NNinput)
        #print i ,output_NN
        
        if output_NN[0,1]>0.6:
            
            j1_LV = ROOT.TLorentzVector()
            j2_LV = ROOT.TLorentzVector()
            
            j1_LV.SetPtEtaPhiE(Pt1_EMJESGSCMu,
                           Eta1_EMJESGSCMu, Phi1_EMJESGSCMu,E1_EMJESGSCMu)
            j2_LV.SetPtEtaPhiE(Pt2_EMJESGSCMu,
                           Eta2_EMJESGSCMu, Phi2_EMJESGSCMu,E2_EMJESGSCMu)
            
            dijet_LV = j1_LV + j2_LV
            Mass = dijet_LV.M()
            
            if process == 'lvbb125': 
                list_hist_M[0].Fill(Mass,tree.eventWeight)
            if process == 'Wbb':
                list_hist_M[1].Fill(Mass,tree.eventWeight)
            if process == 'TTbar':
                list_hist_M[2].Fill(Mass,tree.eventWeight)
            #if process == 'Wbl':
            #    list_hist_M[2].Fill(Mass,tree.eventWeight)
            #if process == 'Wcc':
            #    list_hist_M[3].Fill(Mass,tree.eventWeight)
            #if process == 'Wcl':
            #    list_hist_M[4].Fill(Mass,tree.eventWeight)
            #if process == 'Wll':
            #    list_hist_M[5].Fill(Mass,tree.eventWeight)
            #if process == 'TTbar':
            #    list_hist_M[6].Fill(Mass,tree.eventWeight)
            #if process == 'WW':
            #    list_hist_M[7].Fill(Mass,tree.eventWeight)
            #if process == 'WZ':
            #    list_hist_M[8].Fill(Mass,tree.eventWeight)
            #if process == 'ZZ':
            #    list_hist_M[9].Fill(Mass,tree.eventWeight)
                
clone_list_M = []
for hist in list_hist_M:
    newhist = hist.Clone()
    clone_list_M.append(newhist)
    
#HistogramFunctions.Draw1DHists(list_hist_M,output_path)
HistogramFunctions.DrawList1DHistInCanvas(clone_list_M, "Mass Plot", " Invariant Mass ", "Events", c)        

c

In [ ]:
c.Print(output_path+'/plot_junto_encontro_fisica_com_todos_backgrounds.pdf')

In [ ]:
ROOT.gROOT.Reset()
ROOT.gStyle.SetOptStat(0)

In [ ]:
l = rootnotes.canvas("k", (800, 600))
#ROOT.gROOT.Reset()
ROOT.gStyle.SetOptStat(0)

hs = ROOT.THStack('hs',"")
leg = ROOT.TLegend(0.9,0.4,0.6,0.9)

clone_list_M[1].SetFillStyle(200)
clone_list_M[1].SetFillColor(ROOT.kNone)
clone_list_M[1].SetFillColor(list_color[1])
clone_list_M[1].SetLineColor(ROOT.kBlack)
hs.Add(clone_list_M[1])
leg.AddEntry(clone_list_M[1],"Wbb","f")

clone_list_M[2].SetFillStyle(200)
clone_list_M[2].SetFillColor(ROOT.kNone)
clone_list_M[2].SetFillColor(list_color[2])
clone_list_M[2].SetLineColor(ROOT.kBlack)
hs.Add(clone_list_M[2])
leg.AddEntry(clone_list_M[1],"Wbl","f")

clone_list_M[3].SetFillStyle(200)
clone_list_M[3].SetFillColor(ROOT.kNone)
clone_list_M[3].SetFillColor(list_color[3])
clone_list_M[3].SetLineColor(ROOT.kBlack)
hs.Add(clone_list_M[3])
leg.AddEntry(clone_list_M[3],"Wcc","f")

clone_list_M[4].SetFillStyle(200)
clone_list_M[4].SetFillColor(ROOT.kNone)
clone_list_M[4].SetFillColor(list_color[4])
clone_list_M[4].SetLineColor(ROOT.kBlack)
hs.Add(clone_list_M[4])
leg.AddEntry(clone_list_M[4],"Wcl","f")

clone_list_M[5].SetFillStyle(200)
clone_list_M[5].SetFillColor(ROOT.kNone)
clone_list_M[5].SetFillColor(list_color[5])
clone_list_M[5].SetLineColor(ROOT.kBlack)
hs.Add(clone_list_M[5])
leg.AddEntry(clone_list_M[5],"Wll","f")

clone_list_M[6].SetFillStyle(200)
clone_list_M[6].SetFillColor(ROOT.kNone)
clone_list_M[6].SetFillColor(list_color[6])
clone_list_M[6].SetLineColor(ROOT.kBlack)
hs.Add(clone_list_M[6])
leg.AddEntry(clone_list_M[6],"TTbar","f")

clone_list_M[7].SetFillStyle(200)
clone_list_M[7].SetFillColor(ROOT.kNone)
clone_list_M[7].SetFillColor(list_color[7])
clone_list_M[7].SetLineColor(ROOT.kBlack)
hs.Add(clone_list_M[7])
leg.AddEntry(clone_list_M[7],"WW","f")

clone_list_M[8].SetFillStyle(200)
clone_list_M[8].SetFillColor(ROOT.kNone)
clone_list_M[8].SetFillColor(list_color[8])
clone_list_M[8].SetLineColor(ROOT.kBlack)
hs.Add(clone_list_M[8])
leg.AddEntry(clone_list_M[8],"WZ","f")

clone_list_M[9].SetFillStyle(200)
clone_list_M[9].SetFillColor(ROOT.kNone)
clone_list_M[9].SetFillColor(list_color[9])
clone_list_M[9].SetLineColor(ROOT.kBlack)
hs.Add(clone_list_M[9])
leg.AddEntry(clone_list_M[9],"ZZ","f")

clone_list_M[0].SetFillStyle(200)
clone_list_M[0].SetFillColor(ROOT.kNone)
clone_list_M[0].SetFillColor(ROOT.kRed)
clone_list_M[0].SetLineColor(ROOT.kBlack)
hs.Add(clone_list_M[0])
leg.AddEntry(clone_list_M[0],"WH125","f")

hs.Draw();
hs.GetXaxis().SetTitle("m_{b#bar{b}} [GeV]");
hs.GetYaxis().SetTitle("Arbitrary Units");
leg.Draw();
#//hs->GetXaxis()->SetRangeUser(50.,150.);
#gPad.Modified();
#q.Print(output_path+'/stacked_ENFPC.pdf')
l

In [ ]:
q = rootnotes.canvas("k", (800, 600))
#ROOT.gROOT.Reset()
ROOT.gStyle.SetOptStat(0)
clone_list_M[1].SetFillStyle(200)
clone_list_M[1].SetFillColor(ROOT.kNone)
clone_list_M[1].SetFillColor(ROOT.kRed)
clone_list_M[1].Draw()
q

In [ ]:
c.Print(output_path+'/predictPlot.pdf')
print list_hist_M[0].GetEntries()
print list_hist_M[1].GetEntries()